In [1]:
from utils import *
from sklearn.datasets import make_friedman1, make_friedman2, make_friedman3
from scipy import optimize
from scipy.stats import spearmanr
from multiprocessing import Pool, cpu_count
from functools import partial
from itertools import repeat
import pandas as pd
depths = (3, 100)
# corrcoef_type = 'p'
# c_val_bounds = 'fixed' #(1e-9, 1e5) 

In [2]:
def runner(
    fried_num, 
    noise, 
    norm,
    c_val_bounds, 
    corrcoef_type,
    ntk_depth, 
    ansiotropic,
    white,
    w_fixed=False,
    opt_bounds = [0.0001, 10],
    opt_resets = 10,
    figs = False,
    diagnostic = False,
    exp_name = ''
    ):

    if type(ntk_depth) == int:
        ntk_depth = (ntk_depth,)

    return_vals = {
        'Dataset' : f'Friedman {fried_num}',
        'Noise': False if noise == 0.0 else True,
        'Norm' : norm,
        'Constant Value Bounds' : 'fixed' if c_val_bounds == 'fixed' else 'unfixed',
        'Corr Type' : 'Pearson ' if corrcoef_type == 'p' else 'Spearman '
        }
    print(
        f"Friedman {fried_num}, Noise: {noise}, Norm: {norm}, cv_bounds: {c_val_bounds}, corrcoef_type: {corrcoef_type}"
    )

    job_name = f"Friedman_{fried_num}_Noise_{noise}_Norm_{norm}_c_val_{return_vals['Constant Value Bounds']}_ansiotropic_{ansiotropic}_white_{white}_w_fixed_{w_fixed}"
    if figs == True and exp_name == '':
        exp_name = f"Friedman_{fried_num}_Noise_{noise}_Norm_{norm}_c_val_{return_vals['Constant Value Bounds']}_ansiotropic_{ansiotropic}_white_{white}_w_fixed_{w_fixed}"

    opt_lower, opt_upper = opt_bounds

    # Datasets
    X_init, y_init = [None, None]
    if fried_num==1:
        X_init, y_init = make_friedman1(noise=noise, random_state=18397425)
    elif fried_num==2:
        X_init, y_init = make_friedman2(noise=noise, random_state=18397425)
    elif fried_num==3:
        X_init, y_init = make_friedman3(noise=noise, random_state=18397425)
    else:
        return Exception("fried_num must be either 1, 2, or 3")
    
    datasets = processing(X_init, y_init)
    X, y, X_train, y_train = [None, datasets['orig'][1], None, datasets['orig train'][1]]

    if norm:
        X = datasets['norm'][0]
        X_train = datasets['norm train'][0]
    else:
        X = datasets['orig'][0]
        X_train = datasets['orig train'][0]



    # Lengthscale Optimization
    def g(ell, gp, residual, typ, p_or_s='p'):
        try:
            gp.set_params(**{'kernel__k2__length_scale': ell})
        except:
            gp.set_params(**{'kernel__k1__k2__length_scale': ell})

        gp.fit(X_train, y_train)
        mean = gp.predict(X)

        if diagnostic == True:
            print(typ, gp.kernel_)

        if p_or_s == 's':
            return -spearmanr(residual, y-mean)[0]
        else:
            return -np.corrcoef((residual)[:, 0], (y-mean)[:, 0])[0, 1]

    for depth in ntk_depth:
        # Kernels
        if noise != 0.0 and white == True:
            ntk = (
                ConstantKernel(
                    constant_value=1,
                    constant_value_bounds=(1e-9, 1e2)
                ) *
                NTK(depth=depth, c=2,
                    bias=1e-1,
                    bias_bounds=(1e-9, 1e0)
                    ) + WhiteKernel(noise_level=1, noise_level_bounds='fixed' if w_fixed else (0.001, 100))
            )
        else:
            ntk = (
                ConstantKernel(
                    constant_value=1,
                    constant_value_bounds=(1e-9, 1e2)
                ) *
                NTK(depth=depth, c=2,
                    bias=1e-1,
                    bias_bounds=(1e-9, 1e0)
                    )
            )

        gp_ntk = GPR(kernel=ntk, alpha=1e-9, normalize_y=True,
                    n_restarts_optimizer=9, random_state=3480795)
        gp_ntk.fit(X_train, y_train)
        print(f'NTK {depth} : ', gp_ntk.kernel_)
        mean_ntk = gp_ntk.predict(X)
            


        if noise != 0.0 and white == True:
            lpk = (
                ConstantKernel(
                    constant_value=0.224**2,
                    constant_value_bounds=c_val_bounds
                ) *
                Matern(
                    nu=1/2,
                    length_scale=np.array([np.random.rand()*np.random.randint(1, 1000) 
                                        for i in range(0, X.shape[1])]) if ansiotropic else 1,
                    length_scale_bounds='fixed'  # (1e-1, 1e3),
                ) + 
                WhiteKernel(
                    noise_level=gp_ntk.kernel_.get_params()['k2__noise_level'] if w_fixed else 1, 
                    noise_level_bounds='fixed' if w_fixed else (0.001, 100))
            )
        else:
            lpk = (
                ConstantKernel(
                    constant_value=0.224**2,
                    constant_value_bounds=c_val_bounds
                ) *
                Matern(
                    nu=1/2,
                    # anisotropic true -> trying to fit product of laplace kernels with different lengthscales
                    length_scale=np.array([np.random.rand()*np.random.randint(1, 1000)
                                        for i in range(0, X.shape[1])]) if ansiotropic else 1,
                    # length_scale=1,
                    length_scale_bounds='fixed'  # (1e-1, 1e3),
                )
            )

        gp_lpk = GPR(kernel=lpk, alpha=1e-9, normalize_y=True,
                    n_restarts_optimizer=0, random_state=3480795)

        if ansiotropic:
            ell_lpk = optimize.minimize(g, np.array([np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]), args=(
                gp_lpk, y-mean_ntk, f'LPK {depth}:', corrcoef_type), bounds=tuple((0.00001, 10) for n in range(0, X.shape[1])))
            for i in range(0, opt_resets):
                tmp = optimize.minimize(g, np.array([[np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]]), args=(
                    gp_lpk, y-mean_ntk, f'LPK {depth}:', corrcoef_type), bounds=tuple((0.00001, 10**i) for n in range(0, X.shape[1])))
                if tmp.fun < ell_lpk.fun:
                    ell_lpk = tmp

        else:
            ell_lpk = optimize.minimize_scalar(g, args=(
                gp_lpk, y-mean_ntk, f'LPK {depth}:', corrcoef_type), method='bounded', bounds=opt_bounds, options={'maxiter': 10000})
            for i in range(0, opt_resets):
                tmp = optimize.minimize_scalar(g, args=(gp_lpk, y-mean_ntk, f'LPK {depth}:', corrcoef_type),
                                               method='bounded', bounds=[opt_lower, opt_upper*(10**i)], options={'maxiter': 10000})
                if tmp.fun < ell_lpk.fun:
                    ell_lpk = tmp

        try:
            gp_lpk.set_params(**{'kernel__k2__length_scale': ell_lpk.x})
        except:
            gp_lpk.set_params(**{'kernel__k1__k2__length_scale': ell_lpk.x})
        gp_lpk.fit(X_train, y_train)
        mean_lpk_opt = gp_lpk.predict(X)



        # Begin gathering return values
        return_vals[f'lpk_{depth}'] = ell_lpk.x
        return_vals[f'f_lpk_{depth}'] = -ell_lpk.fun

        if corrcoef_type == 's':
            return_vals[f'fp_lpk_{depth}'] = np.corrcoef((y-mean_ntk)[:,0], (y-mean_lpk_opt)[:,0])[0, 1]
        else:
            return_vals[f'fs_lpk_{depth}'] = spearmanr(y-mean_ntk, y-mean_lpk_opt)[0]



        if norm:
            if noise != 0.0 and white == True:
                rbf = (
                    ConstantKernel(
                        constant_value=0.224**2, 
                        constant_value_bounds=c_val_bounds
                    ) * 
                    Matern(
                        nu=np.inf,
                        length_scale=np.array([np.random.rand()*np.random.randint(1, 1000)
                                               for i in range(0, X.shape[1])]) if ansiotropic else 1,
                        length_scale_bounds='fixed'#(1e-1, 1e3), 
                    ) + 
                    WhiteKernel(
                        noise_level=gp_ntk.kernel_.get_params()['k2__noise_level'] if w_fixed else 1,
                        noise_level_bounds='fixed' if w_fixed else (0.001, 100))
                )
            else:
                rbf = (
                    ConstantKernel(
                        constant_value=0.224**2,
                        constant_value_bounds=c_val_bounds
                    ) *
                    Matern(
                        nu=np.inf,
                        length_scale=np.array([np.random.rand()*np.random.randint(1, 1000)
                                               for i in range(0, X.shape[1])]) if ansiotropic else 1,
                        length_scale_bounds='fixed',  # (1e-1, 1e3),
                    )
                )

            gp_rbf = GPR(kernel=rbf, alpha=1e-9, normalize_y=True, n_restarts_optimizer=0, random_state=3480795)

            if ansiotropic:
                ell_rbf = optimize.minimize(g, np.array([np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]), args=(
                    gp_rbf, y-mean_ntk, f'RBF {depth}:', corrcoef_type), bounds=tuple((0.00001, 10) for n in range(0, X.shape[1])))
                for i in range(0, opt_resets):
                    tmp = optimize.minimize(g, np.array([[np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]]), args=(
                        gp_rbf, y-mean_ntk, f'RBF {depth}:', corrcoef_type), bounds=tuple((0.00001, 10**i) for n in range(0, X.shape[1])))
                    if tmp.fun < ell_rbf.fun:
                        ell_rbf = tmp

            else:
                ell_rbf = optimize.minimize_scalar(g, args=(
                    gp_rbf, y-mean_ntk, f'RBF {depth}:', corrcoef_type), method='bounded', bounds=opt_bounds, options={'maxiter': 10000})
                for i in range(0, opt_resets):
                    tmp = optimize.minimize_scalar(g, args=(gp_rbf, y-mean_ntk, f'RBF {depth}:', corrcoef_type),
                                                method='bounded', bounds=[opt_lower, opt_upper*(10**i)], options={'maxiter': 10000})
                    if tmp.fun < ell_rbf.fun:
                        ell_rbf = tmp

            try:
                gp_rbf.set_params(**{'kernel__k2__length_scale': ell_rbf.x})
            except:
                gp_rbf.set_params(**{'kernel__k1__k2__length_scale': ell_rbf.x})
            gp_rbf.fit(X_train, y_train)
            mean_rbf_opt = gp_rbf.predict(X)

            # Continue gathering return values
            return_vals[f'rbf_{depth}'] = ell_rbf.x
            return_vals[f'f_rbf_{depth}'] = -ell_rbf.fun

            if corrcoef_type == 's':
                return_vals[f'fp_rbf_{depth}'] = np.corrcoef((y-mean_ntk)[:,0], (y-mean_rbf_opt)[:,0])[0, 1]
            else:
                return_vals[f'fs_rbf_{depth}'] = spearmanr(y-mean_ntk, y-mean_rbf_opt)[0]



            if figs:
                # residuals vs y_hat (LPK and NTK SEPARATE)
                fig, axes = plt.subplots(nrows=1, ncols=3 if norm else 2, sharex=True,sharey=True, figsize=(12 if norm else 8, 4))
                axes[0].scatter(y-mean_ntk, mean_ntk)
                axes[0].set_title('NTK')
                axes[1].scatter(y-mean_lpk_opt, mean_lpk_opt)
                axes[1].set_title('LPK')
                if norm:
                    axes[2].scatter(y-mean_rbf_opt, mean_rbf_opt)
                    axes[2].set_title('RBF')
                fig.supxlabel('Residuals')
                fig.supylabel(r'$\hat{y}$')
                fig.suptitle(r"Residuals vs. $\hat{y}$ for " + f"NTK Depth = {depth}")
                fig.savefig(f'./{exp_name}/res_vs_yhat_depth_{depth}_{job_name}.svg', transparent=True, bbox_inches='tight')
                
                # residuals vs residuals (LPK vs NTK)
                fig, axes = plt.subplots(nrows=1, ncols=2 if norm else 1, sharex=True, sharey=True, figsize=(12 if norm else 8, 4))
                axes[0].scatter(y-mean_lpk_opt, y-mean_ntk)
                axes[0].set_xlabel("LPK Residuals")
                if norm:
                    axes[1].scatter(y-mean_rbf_opt, y-mean_ntk)
                    axes[1].set_xlabel("RBF Residuals")
                fig.supylabel("NTK Residuals")
                fig.suptitle(f"Residuals vs. Residuals for NTK Depth = {depth}")
                fig.savefig(f'./{exp_name}/res_vs_res_depth_{depth}_{job_name}.svg', transparent=True, bbox_inches='tight')

                # y_hat vs y_hat (LPK vs NTK)
                fig, axes = plt.subplots(nrows=1, ncols=2 if norm else 1, sharex=True, sharey=True, figsize=(12 if norm else 8, 4))
                axes[0].scatter(mean_lpk_opt, mean_ntk)
                axes[0].set_xlabel(r'$y_{LPK}$')
                if norm:
                    axes[1].scatter(mean_rbf_opt, mean_ntk)
                    axes[1].set_xlabel(r'$y_{RBF}$')
                fig.supylabel(r'$y_{NTK}$')
                fig.suptitle(f"Predicted mean comparison for NTK Depth = {depth}")
                fig.savefig(f'./{exp_name}/y_vs_y_depth_{depth}_{job_name}.svg', transparent=True, bbox_inches='tight')

    return return_vals

In [8]:
# %%capture
depths=(3,100)
params = [
    [1, 0.15, False, 'fixed', 'p'],
    [2, 0.15, False, 'fixed', 'p']
]

kparams = {'ntk_depth': depths, 'ansiotropic': False, 'white': True, 'w_fixed': False, 'diagnostic': True}

# test = []
# for param in params:
#     test.append(runner(*param, **kparams))

with Pool(processes=cpu_count()) as p:
    test = p.starmap(
        partial(runner, **kparams), params
    )

# test = runner(*[        1,   0.15, True,      (1e-9, 1e5),  100,           'p'], 
#     diagnostic=False, ansiotropic=True, white=False)

Friedman 2, Noise: 0.0, Norm: False, cv_bounds: fixed, corrcoef_type: p
Friedman 1, Noise: 0.0, Norm: False, cv_bounds: fixed, corrcoef_type: p
NTK 3 :  0.0113**2 * NTK(depth=3, c=2.000, bias=0.056)
LPK 3: 0.224**2 * Matern(length_scale=3.82, nu=0.5)
LPK 3: 0.224**2 * Matern(length_scale=6.18, nu=0.5)
LPK 3: 0.224**2 * Matern(length_scale=7.64, nu=0.5)
LPK 3: 0.224**2 * Matern(length_scale=8.54, nu=0.5)
LPK 3: 0.224**2 * Matern(length_scale=9.1, nu=0.5)
LPK 3: 0.224**2 * Matern(length_scale=9.44, nu=0.5)
LPK 3: 0.224**2 * Matern(length_scale=9.66, nu=0.5)
LPK 3: 

/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gaussian_process/_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


0.224**2 * Matern(length_scale=9.79, nu=0.5)
LPK 3: 0.224**2 * Matern(length_scale=9.87, nu=0.5)
LPK 3: 0.224**2 * Matern(length_scale=9.92, nu=0.5)
NTK 3 : LPK 3: 0.224**2 * Matern(length_scale=9.95, nu=0.5) 
0.441**2 * NTK(depth=3, c=2.000, bias=0.353)
LPK 3: 0.224**2 * Matern(length_scale=9.97, nu=0.5)
LPK 3:LPK 3:  0.224**2 * Matern(length_scale=3.82, nu=0.5)
0.224**2 * Matern(length_scale=9.98, nu=0.5)LPK 3: 0.224**2 * Matern(length_scale=6.18, nu=0.5)

LPK 3:LPK 3: 0.224**2 * Matern(length_scale=9.99, nu=0.5) 0.224**2 * Matern(length_scale=2.36, nu=0.5)

LPK 3:LPK 3:  0.224**2 * Matern(length_scale=1.46, nu=0.5)
0.224**2 * Matern(length_scale=9.99, nu=0.5)
LPK 3:LPK 3:  0.224**2 * Matern(length_scale=10, nu=0.5)
0.224**2 * Matern(length_scale=2.84, nu=0.5)
LPK 3:LPK 3: 0.224**2 * Matern(length_scale=10, nu=0.5) 
0.224**2 * Matern(length_scale=2.02, nu=0.5)
LPK 3:LPK 3: 0.224**2 * Matern(length_scale=2.41, nu=0.5)
LPK 3: 0.224**2 * Matern(length_scale=10, nu=0.5) 
0.224**2 * Mater

In [4]:
test

[{'Dataset': 'Friedman 1',
  'Noise': False,
  'Norm': False,
  'Constant Value Bounds': 'fixed',
  'Corr Type': 'Pearson ',
  'lpk_3': 2.342221503285901,
  'f_lpk_3': 0.8883573030438279,
  'fs_lpk_3': 0.8254305430543053,
  'lpk_100': 0.23322660322706876,
  'f_lpk_100': 0.9956018205477503,
  'fs_lpk_100': 0.9329972997299729},
 {'Dataset': 'Friedman 2',
  'Noise': False,
  'Norm': False,
  'Constant Value Bounds': 'fixed',
  'Corr Type': 'Pearson ',
  'lpk_3': 30.487682918826152,
  'f_lpk_3': 0.658529378793874,
  'fs_lpk_3': 0.5878547854785479,
  'lpk_100': 28.20266662990849,
  'f_lpk_100': 0.647673153812719,
  'fs_lpk_100': 0.5200120012001199}]

## Experiment 0: Constant Value Optimization

In [9]:
params = [
    #fried_num  noise   norm c_val_bounds  
    [        1,   0.0, False,      'fixed', 'p'], # 0
    [        1,   0.0, False,  (1e-9, 1e5), 'p'], # 1
    [        1,   0.0,  True,      'fixed', 'p'], # 2
    [        1,   0.0,  True,  (1e-9, 1e5), 'p'], # 3
    [        1,  0.15, False,      'fixed', 'p'], # 4 
    [        1,  0.15, False,  (1e-9, 1e5), 'p'], # 5
    [        1,  0.15,  True,      'fixed', 'p'], # 6
    [        1,  0.15,  True,  (1e-9, 1e5), 'p'], # 7
    
    [        1,   0.0, False,      'fixed', 's'], # 8
    [        1,   0.0, False,  (1e-9, 1e5), 's'], # 9
    [        1,   0.0,  True,      'fixed', 's'], # 10
    [        1,   0.0,  True,  (1e-9, 1e5), 's'], # 11
    [        1,  0.15, False,      'fixed', 's'], # 12
    [        1,  0.15, False,  (1e-9, 1e5), 's'], # 13
    [        1,  0.15,  True,      'fixed', 's'], # 14
    [        1,  0.15,  True,  (1e-9, 1e5), 's'], # 15


    [        2,   0.0, False,      'fixed', 'p'], # 16
    [        2,   0.0, False,  (1e-9, 1e5), 'p'], # 17
    [        2,   0.0,  True,      'fixed', 'p'], # 18
    [        2,   0.0,  True,  (1e-9, 1e5), 'p'], # 19
    [        2,  0.15, False,      'fixed', 'p'], # 20
    [        2,  0.15, False,  (1e-9, 1e5), 'p'], # 21
    [        2,  0.15,  True,      'fixed', 'p'], # 22
    [        2,  0.15,  True,  (1e-9, 1e5), 'p'], # 23

    [        2,   0.0, False,      'fixed', 's'], # 24
    [        2,   0.0, False,  (1e-9, 1e5), 's'], # 25
    [        2,   0.0,  True,      'fixed', 's'], # 26
    [        2,   0.0,  True,  (1e-9, 1e5), 's'], # 27
    [        2,  0.15, False,      'fixed', 's'], # 28
    [        2,  0.15, False,  (1e-9, 1e5), 's'], # 29
    [        2,  0.15,  True,      'fixed', 's'], # 30
    [        2,  0.15,  True,  (1e-9, 1e5), 's'], # 31


    [        3,   0.0, False,      'fixed', 'p'], # 32
    [        3,   0.0, False,  (1e-9, 1e5), 'p'], # 33
    [        3,   0.0,  True,      'fixed', 'p'], # 34
    [        3,   0.0,  True,  (1e-9, 1e5), 'p'], # 35
    [        3,  0.15, False,      'fixed', 'p'], # 36
    [        3,  0.15, False,  (1e-9, 1e5), 'p'], # 37
    [        3,  0.15,  True,      'fixed', 'p'], # 38
    [        3,  0.15,  True,  (1e-9, 1e5), 'p'], # 39
    
    [        3,   0.0, False,      'fixed', 's'], # 40
    [        3,   0.0, False,  (1e-9, 1e5), 's'], # 41
    [        3,   0.0,  True,      'fixed', 's'], # 42
    [        3,   0.0,  True,  (1e-9, 1e5), 's'], # 43
    [        3,  0.15, False,      'fixed', 's'], # 44
    [        3,  0.15, False,  (1e-9, 1e5), 's'], # 45
    [        3,  0.15,  True,      'fixed', 's'], # 46
    [        3,  0.15,  True,  (1e-9, 1e5), 's'], # 47
]

depths = (3, 100)

In [10]:
arrays_lpk = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [False, True],  # Noise
    [False, True],  # Norm
    depths,
    ['fixed', 'unfixed']
]

arrays_rbf = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [False, True],  # Noise
    depths,
    ['fixed', 'unfixed']
]

index_lpk = pd.MultiIndex.from_product(
    arrays_lpk, names=["Dataset", "Noise", "Norm", "Depth", "Constant Value Bounds"])

index_rbf = pd.MultiIndex.from_product(
    arrays_rbf, names=["Dataset", "Noise", "Depth", "Constant Value Bounds"])


df_lpk = pd.DataFrame(
    index=index_lpk,
    columns=['Pearson X', 'Spearman X', 'Pearson f(X)', 'Spearman f(Xp)', 'Spearman f(X)', 'Pearson f(Xs)'])

df_rbf = pd.DataFrame(
    index=index_rbf,
    columns=['Pearson X', 'Spearman X', 'Pearson f(X)', 'Spearman f(Xp)', 'Spearman f(X)', 'Pearson f(Xs)'])


In [5]:
%%capture
kparams = {'ntk_depth': depths, 'ansiotropic': False, 'white': False}
with Pool(processes=cpu_count()) as p:
    results = p.starmap(partial(runner, **kparams), params)

In [11]:
for out in results:
    for depth in depths:

        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['lpk_' + str(depth)]
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_lpk_' + str(depth)]

        if out['Corr Type'] == 'Pearson ':
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_lpk_' + str(depth)]
        else:
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_lpk_' + str(depth)]

        if out["Norm"] == True:
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['rbf_' + str(depth)]
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_rbf_' + str(depth)]

            if out['Corr Type'] == 'Pearson ':
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_rbf_' + str(depth)]
            else:
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_rbf_' + str(depth)]

In [12]:
df_lpk_const, df_rbf_const = df_lpk, df_rbf

In [13]:
df_lpk_const

Pearson X  \
Dataset    Noise Norm  Depth Constant Value Bounds                      
Friedman 1 False False 3     fixed                           2.342222   
                             unfixed                         2.342221   
                       100   fixed                           0.233227   
                             unfixed                         0.233227   
                 True  3     fixed                           4.527338   
                             unfixed                         4.527338   
                       100   fixed                           0.119707   
                             unfixed                         0.119707   
           True  False 3     fixed                           2.293013   
                             unfixed                         2.293012   
                       100   fixed                           0.233795   
                             unfixed                         0.233795   
                 True  3     fixed                            4.49483   
                             unfixed                         4.494827   
                       100   fixed                            0.11966   
                             unfixed                          0.11966   
Friedman 2 False False 3     fixed                          30.487683   
                             unfixed                        30.487683   
                       100   fixed                          28.202667   
                             unfixed                        28.202666   
                 True  3     fixed                           21.45487   
                             unfixed                     34127.117312   
                       100   fixed                           0.078187   
                             unfixed                         0.078187   
           True  False 3     fixed                          30.498404   
                             unfixed                        30.498404   
                       100   fixed                           28.21029   
                             unfixed                         28.21029   
                 True  3     fixed                          21.458308   
                             unfixed                      30144.33585   
                       100   fixed                           0.078163   
                             unfixed                         0.078164   
Friedman 3 False False 3     fixed                   108553386.311641   
                             unfixed                8568493249.588059   
                       100   fixed                   119623366.746353   
                             unfixed                7013795880.405347   
                 True  3     fixed                          50.589027   
                             unfixed                     88106.044562   
                       100   fixed                           0.079075   
                             unfixed                         0.079075   
           True  False 3     fixed                          76.177132   
                             unfixed                        76.177132   
                       100   fixed                          55.908955   
                             unfixed                        55.908954   
                 True  3     fixed                          13.000966   
                             unfixed                     11915.886674   
                       100   fixed                           0.075137   
                             unfixed                         0.075137   

                                                           Spearman X  \
Dataset    Noise Norm  Depth Constant Value Bounds                      
Friedman 1 False False 3     fixed                    12127241.415436   
                             unfixed                 733118736.906666   
                       100   fixed                           0.238627   
                             unfixed        

In [14]:
df_rbf_const

Pearson X      Spearman X  \
Dataset    Noise Depth Constant Value Bounds                                   
Friedman 1 False 3     fixed                        0.333568         0.31932   
                       unfixed                      0.333569         0.31932   
                 100   fixed                        0.134175        0.155386   
                       unfixed                      0.134175        0.155386   
           True  3     fixed                        0.333082        0.340142   
                       unfixed                      0.333082        0.340142   
                 100   fixed                        0.133935        0.148435   
                       unfixed                      0.133935        0.148435   
Friedman 2 False 3     fixed                       70.993551      114.218861   
                       unfixed                    281.910527   373145.786693   
                 100   fixed                       74.875531      114.078093   
                       unfixed                  59211.960861  3474362.698772   
           True  3     fixed                       71.092862       39.400864   
                       unfixed                  37890.882501    53631.235075   
                 100   fixed                       74.904212       39.406189   
                       unfixed                    763.931974   381966.506523   
Friedman 3 False 3     fixed                       48.641787       58.361671   
                       unfixed                  65767.237334    76405.318461   
                 100   fixed                       46.639129     2179.500977   
                       unfixed                  57869.300662  4551426.509869   
           True  3     fixed                      709.437272      243.276637   
                       unfixed                 381970.002972  2439071.354276   
                 100   fixed                      680.902311     1567.715237   
                       unfixed                2605981.073063  2327363.049454   

                                             Pearson f(X) Spearman f(Xp)  \
Dataset    Noise Depth Constant Value Bounds                               
Friedman 1 False 3     fixed                     0.967224       0.912895   
                       unfixed                   0.967224       0.912895   
                 100   fixed                     0.997816       0.966421   
                       unfixed                   0.997816       0.966421   
           True  3     fixed                     0.965526       0.912703   
                       unfixed                   0.965526       0.912703   
                 100   fixed                     0.997726       0.949499   
                       unfixed                   0.997726       0.949499   
Friedman 2 False 3     fixed                     0.586039       0.663678   
                       unfixed                   0.599684       0.651449   
                 100   fixed                     0.590019       0.663306   
                       unfixed                   0.601359       0.672883   
           True  3     fixed                     0.585987        0.66291   
                       unfixed                   0.596987       0.672679   
                 100   fixed                      0.58997       0.663714   
                       unfixed                   0.596931       0.656406   
Friedman 3 False 3     fixed                     0.820542       0.641272   
                       unfixed                   0.825933       0.643348   
                 100   fixed                      0.82306       0.615446   
                       unfixed                    0.83097       0.614509   
           True  3     fixed                     0.671422       0.721224   
                       unfixed                   0.683505       0.725617   
                 100   fixed                     0.677441       0.724596   
                       unfixed                    0.69447       0.7502

## Experiment 1, 1.5 - Noise optimization

In [6]:
params = [
    #fried_num  noise   norm c_val_bounds  
    [        1,  0.15, False,      'fixed', 'p'], # 4 
    [        1,  0.15, False,  (1e-9, 1e5), 'p'], # 5
    [        1,  0.15,  True,      'fixed', 'p'], # 6
    [        1,  0.15,  True,  (1e-9, 1e5), 'p'], # 7
    
    [        1,  0.15, False,      'fixed', 's'], # 12
    [        1,  0.15, False,  (1e-9, 1e5), 's'], # 13
    [        1,  0.15,  True,      'fixed', 's'], # 14
    [        1,  0.15,  True,  (1e-9, 1e5), 's'], # 15


    [        2,  0.15, False,      'fixed', 'p'], # 20
    [        2,  0.15, False,  (1e-9, 1e5), 'p'], # 21
    [        2,  0.15,  True,      'fixed', 'p'], # 22
    [        2,  0.15,  True,  (1e-9, 1e5), 'p'], # 23

    [        2,  0.15, False,      'fixed', 's'], # 28
    [        2,  0.15, False,  (1e-9, 1e5), 's'], # 29
    [        2,  0.15,  True,      'fixed', 's'], # 30
    [        2,  0.15,  True,  (1e-9, 1e5), 's'], # 31


    [        3,  0.15, False,      'fixed', 'p'], # 36
    [        3,  0.15, False,  (1e-9, 1e5), 'p'], # 37
    [        3,  0.15,  True,      'fixed', 'p'], # 38
    [        3,  0.15,  True,  (1e-9, 1e5), 'p'], # 39
    
    [        3,  0.15, False,      'fixed', 's'], # 44
    [        3,  0.15, False,  (1e-9, 1e5), 's'], # 45
    [        3,  0.15,  True,      'fixed', 's'], # 46
    [        3,  0.15,  True,  (1e-9, 1e5), 's'], # 47
]

In [7]:
arrays_lpk = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [True],  # Noise
    [False, True],  # Norm
    depths,
    ['fixed', 'unfixed']
]

arrays_rbf = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [True],  # Noise
    depths,
    ['fixed', 'unfixed']
]

index_lpk = pd.MultiIndex.from_product(
    arrays_lpk, names=["Dataset", "Noise", "Norm", "Depth", "Constant Value Bounds"])

index_rbf = pd.MultiIndex.from_product(
    arrays_rbf, names=["Dataset", "Noise", "Depth", "Constant Value Bounds"])


df_lpk = pd.DataFrame(
    index=index_lpk,
    columns=['Pearson X', 'Spearman X', 'Pearson f(X)', 'Spearman f(Xp)', 'Spearman f(X)', 'Pearson f(Xs)'])

df_rbf = pd.DataFrame(
    index=index_rbf,
    columns=['Pearson X', 'Spearman X', 'Pearson f(X)', 'Spearman f(Xp)', 'Spearman f(X)', 'Pearson f(Xs)'])


Pt. 1: No white kernel

In [28]:
%%capture
kparams = {'ntk_depth': depths, 'ansiotropic': False, 'white': False}
with Pool(processes=cpu_count()) as p:
    results = p.starmap(partial(runner, **kparams), params)

In [29]:
for out in results:
    for depth in depths:

        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['lpk_' + str(depth)]
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_lpk_' + str(depth)]

        if out['Corr Type'] == 'Pearson ':
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_lpk_' + str(depth)]
        else:
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_lpk_' + str(depth)]

        if out["Norm"] == True:
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['rbf_' + str(depth)]
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_rbf_' + str(depth)]

            if out['Corr Type'] == 'Pearson ':
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_rbf_' + str(depth)]
            else:
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_rbf_' + str(depth)]

In [30]:
df_lpk_no_noise, df_rbf_no_noise = df_lpk, df_rbf

In [31]:
df_lpk_no_noise

Pearson X  \
Dataset    Noise Norm  Depth Constant Value Bounds                 
Friedman 1 True  False 3     fixed                      2.293013   
                             unfixed                    2.293012   
                       100   fixed                      0.233795   
                             unfixed                    0.233795   
                 True  3     fixed                       4.49483   
                             unfixed                    4.494827   
                       100   fixed                       0.11966   
                             unfixed                     0.11966   
Friedman 2 True  False 3     fixed                     30.498404   
                             unfixed                   30.498404   
                       100   fixed                      28.21029   
                             unfixed                    28.21029   
                 True  3     fixed                     21.458308   
                             unfixed                 30144.33585   
                       100   fixed                      0.078163   
                             unfixed                    0.078164   
Friedman 3 True  False 3     fixed                     76.177132   
                             unfixed                   76.177132   
                       100   fixed                     55.908955   
                             unfixed                   55.908954   
                 True  3     fixed                     13.000966   
                             unfixed                11915.886674   
                       100   fixed                      0.075137   
                             unfixed                    0.075137   

                                                          Spearman X  \
Dataset    Noise Norm  Depth Constant Value Bounds                     
Friedman 1 True  False 3     fixed                   15459141.187324   
                             unfixed                  38292948.67977   
                       100   fixed                          0.259339   
                             unfixed                        0.259339   
                 True  3     fixed                          5.801071   
                             unfixed                        5.801071   
                       100   fixed                  607323495.583559   
                             unfixed                         2.36076   
Friedman 2 True  False 3     fixed                          32.19287   
                             unfixed                        32.19287   
                       100   fixed                          9.442728   
                             unfixed                        9.442728   
                 True  3     fixed                          6.180382   
                             unfixed                       61.822705   
                       100   fixed                          1.459069   
                             unfixed                       38.194693   
Friedman 3 True  False 3     fixed                         32.728439   
                             unfixed                       32.728439   
                       100   fixed                         32.623864   
                             unfixed                       32.623864   
                 True  3     fixed                          9.999995   
                             unfixed                        39.28504   
                       100   fixed                        854.102004   
                             unfixed                     5760.832108   

                                                   Pearson f(X)  \
Dataset    Noise Norm  Depth Constant Value Bounds                
Friedman 1 True  False 3     fixed                     0.886163   
                             unfixed                   0.886163   
                       100   fixed                     0.995457   
                             unfixed                   0.995457   
           

In [32]:
df_rbf_no_noise

Pearson X      Spearman X  \
Dataset    Noise Depth Constant Value Bounds                                   
Friedman 1 True  3     fixed                        0.333082        0.340142   
                       unfixed                      0.333082        0.340142   
                 100   fixed                        0.133935        0.148435   
                       unfixed                      0.133935        0.148435   
Friedman 2 True  3     fixed                       71.092862       39.400864   
                       unfixed                  37890.882501    53631.235075   
                 100   fixed                       74.904212       39.406189   
                       unfixed                    763.931974   381966.506523   
Friedman 3 True  3     fixed                      709.437272      243.276637   
                       unfixed                 381970.002972  2439071.354276   
                 100   fixed                      680.902311     1567.715237   
                       unfixed                2605981.073063  2327363.049454   

                                             Pearson f(X) Spearman f(Xp)  \
Dataset    Noise Depth Constant Value Bounds                               
Friedman 1 True  3     fixed                     0.965526       0.912703   
                       unfixed                   0.965526       0.912703   
                 100   fixed                     0.997726       0.949499   
                       unfixed                   0.997726       0.949499   
Friedman 2 True  3     fixed                     0.585987        0.66291   
                       unfixed                   0.596987       0.672679   
                 100   fixed                      0.58997       0.663714   
                       unfixed                   0.596931       0.656406   
Friedman 3 True  3     fixed                     0.671422       0.721224   
                       unfixed                   0.683505       0.725617   
                 100   fixed                     0.677441       0.724596   
                       unfixed                    0.69447       0.750219   

                                             Spearman f(X) Pearson f(Xs)  
Dataset    Noise Depth Constant Value Bounds                              
Friedman 1 True  3     fixed                      0.913507      0.965324  
                       unfixed                    0.913507      0.965324  
                 100   fixed                      0.964956      0.997304  
                       unfixed                    0.964956      0.997304  
Friedman 2 True  3     fixed                      0.669091      0.584989  
                       unfixed                     0.68018      0.594913  
                 100   fixed                      0.668083      0.588752  
                       unfixed                     0.67532      0.588319  
Friedman 3 True  3     fixed                      0.727537      0.668973  
                       unfixed                    0.740702      0.678046  
                 100   fixed                      0.730021      0.675244  
                       unfixed                    0.751575      0.685944

Pt. 2: White Noise but FIXED

In [9]:
%%capture
depths = (3, 100)
kparams = {'ntk_depth': depths, 'ansiotropic': False, 'white': True, 'w_fixed': True}
with Pool(processes=cpu_count()) as p:
    results = p.starmap(partial(runner, **kparams), params)

Bad pipe message: %s [b"\xc0\x81SE\x06\xf1\xac\xae\xcd\x1dDm\xf6K\x0e\xde\n\xd5 \xcf\x8fum\xf9\x83sB\x80^r\xee\xed\x11\xd3RTa\xad\xc9\xdbt\xa6\x08\xf8\x82NWu'P}\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t", b'7.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00']
Bad pipe message: %s [b'\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01']
Bad pipe message: %s [b"v\t\xf5\xe0\xd1>\xed\xa1;GJ\xd0O5x\xbd\xee\xf5 \x8c\xa66Mn!\x8e\x8b\x0e\xca\xea\xe4\xbc\xa4\x08\xc9\xc5V'\x8c\xbf\n\xa3U\x91\xf4\x05v\x8a\xfe\xc3\n\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06

In [10]:
for out in results:
    for depth in depths:

        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['lpk_' + str(depth)]
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_lpk_' + str(depth)]

        if out['Corr Type'] == 'Pearson ':
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_lpk_' + str(depth)]
        else:
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_lpk_' + str(depth)]

        if out["Norm"] == True:
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['rbf_' + str(depth)]
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_rbf_' + str(depth)]

            if out['Corr Type'] == 'Pearson ':
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_rbf_' + str(depth)]
            else:
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_rbf_' + str(depth)]

In [11]:
df_lpk_noise_fixed = df_lpk
df_rbf_noise_fixed = df_rbf

By optimizing pearson we are not improving spearman.  Furthermore by optimizing spearman, we do not improve pearson... Mostly.

In [12]:
df_lpk_noise_fixed

Pearson X  \
Dataset    Noise Norm  Depth Constant Value Bounds                   
Friedman 1 True  False 3     fixed                        1.071639   
                             unfixed                      0.557225   
                       100   fixed                  8149611.127416   
                             unfixed                     61.376516   
                 True  3     fixed                        0.617496   
                             unfixed                      0.426124   
                       100   fixed                  7977825.794934   
                             unfixed                     76.748025   
Friedman 2 True  False 3     fixed                     3930.440934   
                             unfixed                      5.166595   
                       100   fixed                    55099.575605   
                             unfixed                      2.198148   
                 True  3     fixed                        0.117455   
                             unfixed                       1.84085   
                       100   fixed                        8.760134   
                             unfixed                      1.926519   
Friedman 3 True  False 3     fixed                      521.179961   
                             unfixed                    127.632576   
                       100   fixed                    82446.383041   
                             unfixed                      1.854608   
                 True  3     fixed                        0.166201   
                             unfixed                      0.388739   
                       100   fixed                        4.263131   
                             unfixed                      4.609808   

                                                      Spearman X Pearson f(X)  \
Dataset    Noise Norm  Depth Constant Value Bounds                              
Friedman 1 True  False 3     fixed                      1.403772     0.995711   
                             unfixed                    0.774402     0.996631   
                       100   fixed                     999.99997          1.0   
                             unfixed                    0.083209          1.0   
                 True  3     fixed                      0.688188     0.994468   
                             unfixed                    9.999995     0.998599   
                       100   fixed                     99.999992          1.0   
                             unfixed                    9.999995          1.0   
Friedman 2 True  False 3     fixed                   4304.411928      0.99976   
                             unfixed                   17.868999     0.996807   
                       100   fixed                  60557.282333     0.999996   
                             unfixed                    5.904216     0.999972   
                 True  3     fixed                      0.148691     0.992477   
                             unfixed                    9.999995     0.990891   
                       100   fixed                      11.05768     0.999995   
                             unfixed                    9.999995     0.999977   
Friedman 3 True  False 3     fixed                    733.137561     0.950669   
                             unfixed                  143.286634     0.958525   
                       100   fixed                  68091.655086     0.999999   
                             unfixed                     2.36076     0.999989   
                 True  3     fixed                      0.131661     0.928064   
                             unfixed                    3.093979     0.988041   
                       100   fixed                      3.877282     0.999979   
                             unfixed                    6.180382     0.999864   

                                                   Spearman f(Xp)  \
Dataset    Noise Norm  Depth Constant Value Bounds          

In [13]:
df_rbf_noise_fixed

Pearson X Spearman X  \
Dataset    Noise Depth Constant Value Bounds                           
Friedman 1 True  3     fixed                     0.480355   0.502029   
                       unfixed                    1.54945   9.999995   
                 100   fixed                  1997.665335   6.180382   
                       unfixed                    9.14625   9.999995   
Friedman 2 True  3     fixed                     0.087652   9.999995   
                       unfixed                   0.557636   9.999995   
                 100   fixed                     1.429003   9.999995   
                       unfixed                   0.602711   9.999995   
Friedman 3 True  3     fixed                     0.132203   0.083994   
                       unfixed                   0.108457   9.999995   
                 100   fixed                     1.004752   0.901786   
                       unfixed                   1.078394   9.999995   

                                             Pearson f(X) Spearman f(Xp)  \
Dataset    Noise Depth Constant Value Bounds                               
Friedman 1 True  3     fixed                     0.996606       0.995056   
                       unfixed                    0.99033       0.988755   
                 100   fixed                          1.0            1.0   
                       unfixed                        1.0            1.0   
Friedman 2 True  3     fixed                     0.993683        0.98333   
                       unfixed                   0.990237       0.988083   
                 100   fixed                     0.999989       0.999832   
                       unfixed                   0.999977       0.999796   
Friedman 3 True  3     fixed                     0.934161       0.947507   
                       unfixed                    0.98526       0.989703   
                 100   fixed                     0.999966       0.999904   
                       unfixed                   0.999864        0.99958   

                                             Spearman f(X) Pearson f(Xs)  
Dataset    Noise Depth Constant Value Bounds                              
Friedman 1 True  3     fixed                      0.995236      0.996591  
                       unfixed                    0.988755       0.99033  
                 100   fixed                           1.0      0.999999  
                       unfixed                         1.0           1.0  
Friedman 2 True  3     fixed                      0.948431      0.974762  
                       unfixed                    0.948203       0.97474  
                 100   fixed                      0.999832      0.999977  
                       unfixed                    0.999796      0.999977  
Friedman 3 True  3     fixed                      0.951575      0.931831  
                       unfixed                    0.901638      0.886219  
                 100   fixed                      0.999952      0.999961  
                       unfixed                     0.99958      0.999864

Pt 3: White Noise but OPTIMIZED

In [14]:
%%capture
depths=(3, 100)
kparams = {'ntk_depth': depths, 'ansiotropic': False, 'white': True, 'w_fixed': False}
with Pool(processes=cpu_count()) as p:
    results = p.starmap(partial(runner, **kparams), params)

In [15]:
for out in results:
    for depth in depths:

        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['lpk_' + str(depth)]
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_lpk_' + str(depth)]

        if out['Corr Type'] == 'Pearson ':
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_lpk_' + str(depth)]
        else:
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_lpk_' + str(depth)]

        if out["Norm"] == True:
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['rbf_' + str(depth)]
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_rbf_' + str(depth)]

            if out['Corr Type'] == 'Pearson ':
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_rbf_' + str(depth)]
            else:
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_rbf_' + str(depth)]

In [16]:
df_lpk_noise_optimized, df_rbf_noise_optimized = df_lpk, df_rbf

In [17]:
df_lpk_noise_optimized

Pearson X  \
Dataset    Noise Norm  Depth Constant Value Bounds                   
Friedman 1 True  False 3     fixed                        0.877504   
                             unfixed                      0.983673   
                       100   fixed                  4376125.040017   
                             unfixed                     66.986787   
                 True  3     fixed                        0.545422   
                             unfixed                      3.357961   
                       100   fixed                        0.254847   
                             unfixed                      0.115278   
Friedman 2 True  False 3     fixed                     1326.804385   
                             unfixed                  62634.061937   
                       100   fixed                    20970.215769   
                             unfixed                  62634.063081   
                 True  3     fixed                        0.107636   
                             unfixed                      3.247937   
                       100   fixed                        4.537584   
                             unfixed                      3.247937   
Friedman 3 True  False 3     fixed                      405.736359   
                             unfixed                    164.519408   
                       100   fixed                     75180.79484   
                             unfixed                  55519.450925   
                 True  3     fixed                        0.154682   
                             unfixed                      4.555568   
                       100   fixed                        0.133128   
                             unfixed                      7.782568   

                                                      Spearman X Pearson f(X)  \
Dataset    Noise Norm  Depth Constant Value Bounds                              
Friedman 1 True  False 3     fixed                      3.819726     0.735696   
                             unfixed                     0.76903     0.858571   
                       100   fixed                     999.99997          1.0   
                             unfixed                   99.999992          1.0   
                 True  3     fixed                      0.678438     0.767429   
                             unfixed                    9.999995     0.955343   
                       100   fixed                      0.118688     0.795826   
                             unfixed                    9.999995      0.99508   
Friedman 2 True  False 3     fixed                   1534.883494     0.976213   
                             unfixed                99999.998491     0.956178   
                       100   fixed                  18086.801399     0.999968   
                             unfixed                99999.998491     0.999811   
                 True  3     fixed                      0.135065     0.986655   
                             unfixed                    9.999995     0.961168   
                       100   fixed                      4.934267     0.999983   
                             unfixed                    9.999995      0.99992   
Friedman 3 True  False 3     fixed                    416.647604      0.78376   
                             unfixed                  196.601214     0.906679   
                       100   fixed                  76393.204371     0.999999   
                             unfixed                99999.998491     0.999987   
                 True  3     fixed                      0.058922     0.715744   
                             unfixed                    4.164141      0.98281   
                       100   fixed                      0.060661     0.983064   
                             unfixed                    6.180382     0.955475   

                                                   Spearman f(Xp)  \
Dataset    Noise Norm  Depth Constant Value Bounds          

In [18]:
df_rbf_noise_optimized

Pearson X Spearman X  \
Dataset    Noise Depth Constant Value Bounds                        
Friedman 1 True  3     fixed                  0.454959   0.438926   
                       unfixed                3.290237   9.999995   
                 100   fixed                  0.362754   3.819725   
                       unfixed                3.290237   9.999995   
Friedman 2 True  3     fixed                  0.077629   9.999995   
                       unfixed                 1.36002   9.999995   
                 100   fixed                  1.023712   9.999995   
                       unfixed                1.987786   9.999995   
Friedman 3 True  3     fixed                  0.150608   0.121749   
                       unfixed                2.420776   9.999995   
                 100   fixed                  0.104079   0.087628   
                       unfixed                2.420776   9.999995   

                                             Pearson f(X) Spearman f(Xp)  \
Dataset    Noise Depth Constant Value Bounds                               
Friedman 1 True  3     fixed                     0.781853       0.792199   
                       unfixed                   0.744623        0.75814   
                 100   fixed                     0.796797       0.918404   
                       unfixed                   0.779677       0.912115   
Friedman 2 True  3     fixed                     0.988599       0.972829   
                       unfixed                   0.961168       0.925245   
                 100   fixed                     0.999962        0.99976   
                       unfixed                    0.99992       0.999604   
Friedman 3 True  3     fixed                      0.72255       0.816814   
                       unfixed                   0.647686       0.768125   
                 100   fixed                     0.984606       0.972133   
                       unfixed                   0.955475       0.934929   

                                             Spearman f(X) Pearson f(Xs)  
Dataset    Noise Depth Constant Value Bounds                              
Friedman 1 True  3     fixed                      0.792547      0.781775  
                       unfixed                     0.75814      0.744623  
                 100   fixed                      0.912139      0.780317  
                       unfixed                    0.912115      0.779677  
Friedman 2 True  3     fixed                      0.925521      0.961194  
                       unfixed                    0.925245      0.961167  
                 100   fixed                       0.99964       0.99992  
                       unfixed                    0.999604       0.99992  
Friedman 3 True  3     fixed                      0.820534       0.72178  
                       unfixed                    0.768125      0.647686  
                 100   fixed                      0.972889      0.984367  
                       unfixed                    0.934929      0.955475

rbf lengthscale infinite -> linear (hyperplane)

## Experiment 2 - Ansiotropic LPK/RBF kernels
* Friedman 1 Noisy Norm
* Friedman 2 Noiseless Nonnormed

In [16]:
arrays_lpk = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [False, True], # Noise
    [False, True], # Norm
    [3, 100], # Depths
]

arrays_rbf = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [False, True], # Noise
    [3, 100], # Depths
]

index_lpk = pd.MultiIndex.from_product(
    arrays_lpk, names=["Dataset", "Noise", "Norm", "Depth"])

index_rbf = pd.MultiIndex.from_product(
    arrays_rbf, names=["Dataset", "Noise", "Depth"])


df_lpk = pd.DataFrame(
    index=index_lpk,
    columns=['Pearson f(X)', 'Spearman f(Xp)', 'Spearman f(X)', 'Pearson f(Xs)'])

df_rbf = pd.DataFrame(
    index=index_rbf,
    columns=['Pearson f(X)', 'Spearman f(Xp)', 'Spearman f(X)', 'Pearson f(Xs)'])


In [17]:
df_lpk

Pearson f(X) Spearman f(Xp) Spearman f(X)  \
Dataset    Noise Norm  Depth                                             
Friedman 1 False False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
           True  False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
Friedman 2 False False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
           True  False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
Friedman 3 False False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
           True  False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   

                             Pearson f(Xs)  
Dataset    Noise Norm  Depth                
Friedman 1 False False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN  
           True  False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN  
Friedman 2 False False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN  
           True  False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN  
Friedman 3 False False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN  
           True  False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN

In [18]:
% % capture
figs = []
params = [
    #    fried_num, noise,  norm, c_val_bounds, ntk_depth, corrcoef_type
    [1,   0.0, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 1
    [1,   0.0,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 3
    [1,  0.15, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 5
    [1,  0.15,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 7

    [1,   0.0, False,  (1e-9, 1e5),  [3, 100],           's'],  # 9
    [1,   0.0,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 11
    [1,  0.15, False,  (1e-9, 1e5),  [3, 100],           's'],  # 13
    [1,  0.15,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 15


    [2,   0.0, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 17
    [2,   0.0,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 19
    [2,  0.15, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 21
    [2,  0.15,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 23

    [2,   0.0, False,  (1e-9, 1e5),  [3, 100],           's'],  # 25
    [2,   0.0,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 27
    [2,  0.15, False,  (1e-9, 1e5),  [3, 100],           's'],  # 29
    [2,  0.15,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 31


    [3,   0.0, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 33
    [3,   0.0,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 35
    [3,  0.15, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 37
    [3,  0.15,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 39

    [3,   0.0, False,  (1e-9, 1e5),  [3, 100],           's'],  # 41
    [3,   0.0,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 43
    [3,  0.15, False,  (1e-9, 1e5),  [3, 100],           's'],  # 45
    [3,  0.15,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 47
]

for param in params:
    out = runner(*param, ansiotropic=True, figs=False)

    for depth in param[4]:
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out['Corr Type'] + "X"] = out['lpk_' + depth]
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out['Corr Type'] + "f(X)"] = out['f_lpk_' + depth]

        if out['Corr Type'] == 'Pearson ':
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        "Spearman f(Xp)"] = out['fs_lpk_' + depth]
        else:
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        "Pearson f(Xs)"] = out['fp_lpk_' + depth]

        if out["Norm"] == True:
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out['Corr Type'] + "X"] = out['rbf_' + depth]
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out['Corr Type'] + "f(X)"] = out['f_rbf_' + depth]
            if out['Corr Type'] == 'Pearson ':
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            "Spearman f(Xp)"] = out['fs_rbf_' + depth]
            else:
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            "Pearson f(Xs)"] = out['fp_rbf_' + depth]


SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' on line 42 (1183362044.py, line 43)